# **2021ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Training (mini-batch=64, optimizer=Adam, learning rate=0.001)

Running this code on Google Colab probably passes simple baselines on Kaggle.

Tovpass


## **Download Data**


If the Google drive links are dead, you can download data from [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data), and upload data manually to the workspace.

In [ ]:
tr_path = 'train.csv'  # path to training data
tt_path = 'test.csv'   # path to testing data

!gdown --id '1AtgWdt7mXNBrai0ktxeGx3ZFMPu1B1KM' --output train.csv
!gdown --id '15MCDFuAW5Z4n2yXJg3VbyIAnP4Dz-Tn_' --output test.csv

#如果連結壞掉可以改用下面這組或把資料直接載到自己的電腦裡電腦裡
# !gdown --id '1p6NE6zTGo8sQyLegdr2NzPTNp4xMFwmt' --output train.csv
# !gdown --id '1fFpPPXp54K9c7_93OE4Tdxm4A9pp8bf8' --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1AtgWdt7mXNBrai0ktxeGx3ZFMPu1B1KM
To: /content/train.csv
100% 1.24M/1.24M [00:00<00:00, 78.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=15MCDFuAW5Z4n2yXJg3VbyIAnP4Dz-Tn_
To: /content/test.csv
100% 555k/555k [00:00<00:00, 81.8MB/s]


# **Import Some Packages**

In [ ]:
# For data preprocess
import numpy as np
import csv
import math
import pandas as pd

# **讀取training data**
讀入training data並對之進行preprocess

In [ ]:
def valid(x, y):
	# TODO: try to filter out extreme values ex:PM2.5 > 100
  if y > 100:
    return False
  return True
def parse2train(data, feats):
  x = []
  y = []

  # 用前面9筆資料預測下一筆PM2.5 所以需要-9
  total_length = data.shape[1] - 9
  for i in range(total_length):
    x_tmp = data[feats,i:i+9]
    y_tmp = data[17,i+9] #第第18個feature是是PM2.5
    # TODO: try to filter out extreme values
    if valid(x_tmp, y_tmp):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)
    # END TODO
  # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
  x = np.array(x)
  y = np.array(y)
  return x,y


#**This is our gradient descent algorithm. Adam was implemented.**

In [ ]:
def minibatch(x, y):
    # 打亂data順序
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # 訓練參數以及初始化
    batch_size = 128
    lr = 0.001
    lam = 0.001
    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    iter_time = 1000
    for num in range(iter_time):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)
            loss = y_batch - np.dot(x_batch,w) - bias
            
            # 計算gradient
            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w)
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            w_0 = np.copy(w)
            
            # 更新weight, bias
            w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
            

    return w, bias

In [ ]:
# The features you choose

feats = [17]
data = pd.read_csv('train.csv') #讀入讀入training file
for col in list(data.columns[2:]):
  data[col] = data[col].astype(str).map(lambda x: x.rstrip('x*#A'))
data = data.values
train_data = np.transpose(np.array(np.float64(data)))
train_x, train_y = parse2train(train_data, feats)
x_train = train_x[:(int)(train_x.shape[0]*0.8)]
x_val = train_x[(int)(train_x.shape[0]*0.8+0.5):]
y_train = train_y[:(int)(train_x.shape[0]*0.8)]
y_val = train_y[(int)(train_y.shape[0]*0.8+0.5):]
w, bias = minibatch(x_train, y_train)

In [ ]:
loss = 0
for i in range(int(x_val.shape[0])):
      yhead = (np.dot(np.reshape(w,-1),x_val[i]) + bias)[0]
      loss += (yhead - y_val[i])**2
      if i < 50:
        #print([yhead, y_val[i]])
        pass
loss = np.sqrt(loss/int(x_val.shape[0]))
print(loss)

58.04867240067673


In [ ]:
train_df = pd.DataFrame(x_train)
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,78.0,78.0,78.0,78.0,78.0,78.0,78.0,80.0,76.0,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10
1,78.0,78.0,78.0,78.0,78.0,78.0,80.0,76.0,70.0,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10
2,78.0,78.0,78.0,78.0,78.0,80.0,76.0,70.0,65.0,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10
3,78.0,78.0,78.0,78.0,80.0,76.0,70.0,65.0,62.0,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10
4,78.0,78.0,78.0,80.0,76.0,70.0,65.0,62.0,59.0,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.10,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13925,94.0,94.0,93.0,93.0,92.0,93.0,94.0,94.0,94.0,1.94,1.92,1.90,2.04,2.05,2.07,2.00,1.93,1.93
13926,94.0,93.0,93.0,92.0,93.0,94.0,94.0,94.0,94.0,1.92,1.90,2.04,2.05,2.07,2.00,1.93,1.93,1.91
13927,93.0,93.0,92.0,93.0,94.0,94.0,94.0,94.0,93.0,1.90,2.04,2.05,2.07,2.00,1.93,1.93,1.91,1.87
13928,93.0,92.0,93.0,94.0,94.0,94.0,94.0,93.0,91.0,2.04,2.05,2.07,2.00,1.93,1.93,1.91,1.87,1.88


# **讀入testing data**
請注意testing的時候資料之間是沒有重疊的，如果要改寫這塊的話請務必注意

In [ ]:
def parse2test(data, feats):
  x = []
  y = []

  # 用前面9筆資料預測下一筆PM2.5 所以需要-9
  total_length = data.shape[1] - 9
  for i in range(857):
    x_tmp = data[feats,9 * i: 9 * i + 9]
    x.append(x_tmp.reshape(-1,))
  # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
  x = np.array(x)
  return x


In [ ]:
data = pd.read_csv('test.csv')
data = data.values
test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)

# **預測testing set並寫成並寫成csv檔檔**

In [ ]:
 with open('my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])
    for i in range(int(test_x.shape[0])):
      writer.writerow([i + 1,(np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]])